Shared Memory
=============
In this part we are going to talk about shared memory and how we can use it to
improve performance.

0 In general
------------
Shared memory is a small portion of memory connected to each thread block, which
is controllable by the user. It is used to lessen the amount of reads and writes
to global memory as the latency is around 100 times lower. Also in cases where
you have many local variables, it can also be an advantage to use shared memory
as they could be pushed to global memory.

To use shared memory, you have to mark your variable with `__shared__`, like so
`__shared__ int array[10][10];`. Shared memory can also be allocated dynamically
using the `extern` keyword. But you then have to add an extra argument, when
running the kernel to define how much shared memory you need. This is done using
a named argument called `shared`, where you define how many bytes of shared
memory you need.

1 Matrix transposition
----------------------
In this section we will be looking at matrix transposition. It is a problem
where we will get problems with memory coalescing without using shared memory.
Our first implementation will just be the naïve one, where we will transpose
directly.

```c++
__global__ void matrixtranspose(
    const int *A,
    int *trA,
    ushort colsA,
    ushort rowsA)
{
    int i = blockIdx.x*T + threadIdx.x;
    int j = blockIdx.y*T + threadIdx.y;
    if( j < colsA && i < rowsA ) {
        trA[j * rowsA + i] = A[i * colsA + j];
    }
}
```

As we can see, we will not get a coalesced memory access when writing to global
memory.

In [1]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import math
import time

mod = SourceModule("""
        __global__ void matrixtranspose(
            const int *A,
            int *trA,
            ushort colsA,
            ushort rowsA)
        {
            int i = blockIdx.x*blockDim.x + threadIdx.x;
            int j = blockIdx.y*blockDim.y + threadIdx.y;
            if( j < colsA && i < rowsA ) {
                trA[j * rowsA + i] = A[i * colsA + j];
            }
        }
        """)

width = 25000
height = 25000

a = np.arange(height * width).astype(np.int32)
a.shape = (height, width)

trA = np.empty((width, height)).astype(np.int32)

dim_size = 32
block_size = (dim_size,dim_size,1)
grid_size = (int(math.ceil(height / float(dim_size))),
             int(math.ceil(width / float(dim_size))))

matrixtranspose = mod.get_function("matrixtranspose")
start_time = time.time()
matrixtranspose(
        cuda.In(a),
        cuda.Out(trA),
        np.uint16(width),
        np.uint16(height),
        block=block_size,
        grid=grid_size)
total_time_transpose = time.time() - start_time

print(trA)

[[        0     25000     50000 ... 624925000 624950000 624975000]
 [        1     25001     50001 ... 624925001 624950001 624975001]
 [        2     25002     50002 ... 624925002 624950002 624975002]
 ...
 [    24997     49997     74997 ... 624949997 624974997 624999997]
 [    24998     49998     74998 ... 624949998 624974998 624999998]
 [    24999     49999     74999 ... 624949999 624974999 624999999]]


2 Shared Memory implementation
------------------------------
To get coalesced memory access, we will use shared memory. We will use the
shared memory to save the part of global memory, which is read by the thread
block. We can then use this saved to write to the correct place in another
thread to get coalesced access.

Before we go on, we have to introduce barriers. Barriers are a way to ensure
that all threads, within a thread block, have reached a specific point before
any can continue. This is useful especially when using larger kernels or shared
memory, where we can make sure that every thread in the thread block has come
beyond a specific point. In CUDA we can use a barrier by calling the
`__syncthreads()` function. It is also important to note, that all code must
pass the same barrier at the same time. Using barriers in a different way will
result in undefined behaviour.
![One thread is yet to reach the barrier, so the two others are waiting](barrier.png)
![All threads have reached the barrier, so they now can continue](barrier.png)

To get coalesced access with share memory, we need to use the `blockIdx` to move
our thread blocks. By swapping `blockIdx.x` and `blockIdx.y`, when we calculate
our position, we can simply transpose within the block in shared memory and
write that result to global memory.
![Swapping two thread blocks in a small grid](threadblocks.png)

```c++
#define T 16
__global__ void matrixtranspose(
    const int *A,
    int *trA,
    ushort colsA,
    ushort rowsA)
{
    __shared__ int tile[T][T+1];
    int tidx = threadIdx.x;
    int tidy = threadIdx.y;
    int i = blockIdx.x*T + tidx;
    int j = blockIdx.y*T + tidy;
    if(j < colsA && i < rowsA) {
        tile[tidy][tidx] = A[i * colsA + j];
    }
    __syncthreads();
    i = blockIdx.y*T + threadIdx.x;
    j = blockIdx.x*T + threadIdx.y;
    if(j < colsA && i < rowsA) {
        trA[i * rowsA + j] = tile[tidx][tidy];
    }
}
```

In [ ]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import math
import time

width = 25000
height = 25000

a = np.arange(height * width).astype(np.int32)
a.shape = (height, width)

trA = np.empty((width, height)).astype(np.int32)

dim_size = 32
block_size = (dim_size,dim_size,1)
grid_size = (int(math.ceil(height / float(dim_size))),
             int(math.ceil(width / float(dim_size))))

mod = SourceModule("""
            #define T """ + str(dim_size) + """
            __global__ void matrixtranspose(
                const int *A,
                int *trA,
                ushort colsA,
                ushort rowsA)
            {
                __shared__ int tile[T][T+1];
                int tidx = threadIdx.x;
                int tidy = threadIdx.y;
                int i = blockIdx.x*T + tidx;
                int j = blockIdx.y*T + tidy;
                if(j < colsA && i < rowsA) {
                    tile[tidy][tidx] = A[j * colsA + i];
                }
                __syncthreads();
                i = blockIdx.y*T + tidx;
                j = blockIdx.x*T + tidy;
                if(j < rowsA && i < colsA) {
                    trA[j * rowsA + i] = tile[tidx][tidy];
                }
            }
            """)

matrixtranspose = mod.get_function("matrixtranspose")
start_time = time.time()
matrixtranspose(
        cuda.In(a),
        cuda.Out(trA),
        np.uint16(width),
        np.uint16(height),
        block=block_size,
        grid=grid_size)
total_time_shared = time.time() - start_time

print(trA)

3 Dynamically allocated shared memory implementation
----------------------------------------------------
In this implementation we will use dynamically allocated shared memory instead
of allocating it directly in the kernel. It does not yield any specific
performance benefit to dynamically allocate shared memory. But it will make the
kernel more general and you will need less code to handle changing block sizes.
We will also need to change the way we call our kernel by adding the argument
`shared`, defining the number of bytes needed, to the function call.

```c++
__global__ void matrixtranspose(
    const int *A,
    int *trA,
    ushort colsA,
    ushort rowsA)
{
    extern __shared__ int tile[];
    int sharedIdx = threadIdx.y*blockDim.y + threadIdx.x;
    int i = blockIdx.x*blockDim.x + threadIdx.x;
    int j = blockIdx.y*blockDim.y + threadIdx.y;
    if( j < colsA && i < rowsA ) {
        tile[sharedIdx] = A[i * colsA + j];
    }
    __syncthreads();
    i = blockIdx.y*blockDim.y + threadIdx.x;
    j = blockIdx.x*blockDim.x + threadIdx.y;
    if(j < colsA && i < rowsA) {
        sharedIdx = threadIdx.x*blockDim.x + threadIdx.y;
        trA[i * rowsA + j] = tile[sharedIdx];
    }
}
```

In [1]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import math
import time

mod = SourceModule("""
        __global__ void matrixtranspose(
            const int *A,
            int *trA,
            ushort colsA,
            ushort rowsA)
        {
            extern __shared__ int tile[];
            int tidx = threadIdx.x;
            int tidy = threadIdx.y;
            int sharedIdx = tidy*blockDim.y + tidx;
            int i = blockIdx.x*blockDim.x + tidx;
            int j = blockIdx.y*blockDim.y + tidy;
            if( j < colsA && i < rowsA ) {
                tile[sharedIdx] = A[j * colsA + i];
            }
            __syncthreads();
            i = blockIdx.y*blockDim.y + tidx;
            j = blockIdx.x*blockDim.x + tidy;
            if(j < rowsA && i < colsA) {
                sharedIdx = tidx*blockDim.x + tidy;
                trA[j * rowsA + i] = tile[sharedIdx];
            }
        }
        """)


width = 25000
height = 25000

a = np.arange(height * width).astype(np.int32)
a.shape = (height, width)

trA = np.empty((width, height)).astype(np.int32)

dim_size = 16
block_size = (dim_size,dim_size,1)
grid_size = (int(math.ceil(height / float(dim_size))),
             int(math.ceil(width / float(dim_size))))

matrixtranspose = mod.get_function("matrixtranspose")
start_time = time.time()
matrixtranspose(
        cuda.In(a),
        cuda.Out(trA),
        np.uint16(width),
        np.uint16(height),
        block=block_size,
        grid=grid_size,
        shared=dim_size**2*4)
total_time_dyn_shared = time.time() - start_time

print(trA)

[[        0     25000     50000 ... 624925000 624950000 624975000]
 [        1     25001     50001 ... 624925001 624950001 624975001]
 [        2     25002     50002 ... 624925002 624950002 624975002]
 ...
 [    24997     49997     74997 ... 624949997 624974997 624999997]
 [    24998     49998     74998 ... 624949998 624974998 624999998]
 [    24999     49999     74999 ... 624949999 624974999 624999999]]


In [3]:
import matplotlib.pyplot as plt
names = ["No shared", "Shared", "Dynamic shared"]
values = [total_time_transpose, total_time_shared, total_time_dyn_shared]

fig=plt.figure(figsize=(8, 8), dpi= 80, facecolor='w', edgecolor='k')
plt.title("Timing")
plt.bar(names, values)

plt.show()

NameError: name 'total_time_transpose' is not defined